In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions") \
    .master("spark://f5db43ce3d38:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext